In [ ]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os


In [ ]:
# !pip install -U kaleido # you need to install for the visualization

# using CA Datasets

In [ ]:
# Read the Excel file
# Data_Status: Indicates the status of the data. The value "2020F" suggests that it is a forecast for the year 2020.
# State: Represents the state for which the data is recorded (in this case, "CA" for California).
# MSN: Stands for "Monthly State Names" and refers to the specific energy metric or variable being measured. Examples include ARICD, ARICV, ARTCD, ARTCV, ARTXD, WWTXV, WXICD, WXICV, ZWCDP, ZWHDP.
df = pd.read_csv('Datasets/pr_CA.csv')
df.drop('Data_Status',axis=1,inplace=True)

In [ ]:
df

# Data Satatics

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# transposed_df = df.set_index(['Data_Status', 'State', 'MSN']).T.reset_index()
# # Convert the Timestamp column to datetime
# transposed_df['Timestamp'] = pd.to_datetime(transposed_df['Timestamp'])


# After Transformation of Data

In [ ]:
df# Assuming your DataFrame is called 'df'
df_trans = df.melt(id_vars=['State', 'MSN'], var_name='Year', value_name='Yearly Data')
# df['Year'] = pd.to_datetime(df['Year'], format='%Y')

# Set the 'State', 'MSN', and 'Year' columns as the index
df_trans.set_index(['State', 'MSN', 'Year'], inplace=True)

# Sort the index in ascending order
df_trans.sort_index(inplace=True)

# Print the resulting time series DataFrame
df_trans.reset_index(inplace=True)
df_trans['Year'] = pd.to_datetime(df_trans['Year'], format='%Y')

df_trans.head()


# Modeling the Data

## Using ARIMA Model

In [ ]:
os.makedirs('Plots/Arima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
    for msn in df_trans['MSN'].unique():
        try:
            
            fig = go.Figure()

            print('State : {} and MSN : {}'.format(State,msn))
            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[(df_trans['State'] == State) & (
                df_trans['MSN'] == msn)][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Energy Consumption Forecast State using ARIMA : {State} : MSN : {msn} ',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            print(State,msn)
            fig.write_image(f'Plots/Arima_results_plots/{State}_{msn}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))


## Using Sarima

In [11]:
os.makedirs('Plots/Sarima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
    for msn in df_trans['MSN'].unique():
        try:
            
            fig = go.Figure()

            print('State : {} and MSN : {}'.format(State,msn))
            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[(df_trans['State'] == State) & (
                df_trans['MSN'] == msn)][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=True)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Energy Consumption Forecast State using SARIMA : {State} : MSN : {msn} ',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            print(State,msn)
            fig.write_image(f'Plots/Sarima_results_plots/{State}_{msn}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        break

State : CA and MSN : ARICD
Mean Absolute Error (MAE): 1.7
Mean Squared Error (MSE): 3.99
Mean Absolute Percentage Error (MAPE): 15.68


CA ARICD
